# Data Mining (KEN4113) #
## Lab 3: Classification ##

Konstantin Sandfort
i6255681

In [163]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn import tree
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold
from pandas import CategoricalDtype

In [133]:
# Import data

# Training Data
data_train = pd.read_csv('caravan.csv')
y_train = data_train['CARAVAN POLICY']
data_train = data_train.drop(columns=['CARAVAN POLICY'])

# Test Data
data_test = pd.read_csv('caravanTest.csv')
y_test = data_test['CARAVAN POLICY']
data_test = data_test.drop(columns=['CARAVAN POLICY'])

pd.set_option('display.max_columns', None)
print(data_test.head(10))

   Customer Subtype  Number of houses  Avg size household  Avg Age  \
0                33                 1                   4        2   
1                 6                 1                   3        2   
2                39                 1                   3        3   
3                 9                 1                   2        3   
4                31                 1                   2        4   
5                30                 1                   2        4   
6                35                 1                   2        4   
7                 6                 1                   3        3   
8                 4                 1                   2        4   
9                10                 1                   4        2   

   Customer main type  Roman catholic  Protestant  Other religion  \
0                   8               0           6               0   
1                   2               0           5               0   
2                   9 

In [159]:
# --- Step 1: Preprocessing ---

# One-hot-encoding of l0 and l2
def ohe(df, col_name, label):
    cols = pd.get_dummies(df,
                          prefix=label,
                          dtype=int,
                          columns=[col_name],
                          drop_first=False)
    return cols

data_train_ohe = ohe(data_train, 'Customer Subtype', 'l0')
data_train_ohe = ohe(data_train_ohe, 'Customer main type', 'l2')

data_test_ohe = ohe(data_test, 'Customer Subtype', 'l0')
data_test_ohe = ohe(data_test_ohe, 'Customer main type', 'l2')

# Standardize data (scaling and centering)
data_train_ohe = preprocessing.StandardScaler().fit_transform(data_train_ohe)
data_test_ohe = preprocessing.StandardScaler().fit_transform(data_test_ohe)

In [ ]:
# Model validation

%matplotlib inline
con_mat = confusion_matrix(y_test, y_pred)

con_mat = pd.DataFrame(con_mat, columns=['Actual Rain', 'Actual No_Rain'], 
                                 index=['Predict Rain', 'Predict No_Rain'])

sns.heatmap(con_mat, annot=True, fmt='d', cmap='PuBu')

In [160]:
# Model 1: Logistical Regression with all features

log_reg_v1_model = LogisticRegression(verbose=False, penalty=None)
log_reg_v1_model.fit(X=data_train_ohe, y=y_train)
score_1 = log_reg_v1_model.score(X=data_test_ohe, y=y_test)
print(f'Model 1 accuracy score: {score_1}')

Model 1 accuracy score: 0.94025


/home/konstantin/PycharmProjects/data_mining_course/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [162]:
# --- Step 2: Feature Selection
# Univariate Feature Selection

def univariate_feature_selection(df, y, k_f):
    df_new = SelectKBest(f_classif, k=k_f).fit_transform(df, y)
    return df_new

ufs_train = univariate_feature_selection(data_train_ohe, y_train, 15)
ufs_test = univariate_feature_selection(data_test_ohe, y_test, 15)
print(ufs_test)

log_reg_v2_model = LogisticRegression(verbose=False, penalty=None)
log_reg_v2_model.fit(X=ufs_train, y=y_train)

score_2 = log_reg_v2_model.score(X=ufs_test, y=y_test)
print(f'Model 1 accuracy score: {score_2}')

[[ 0.28675066  0.61238697 -1.00611988 ...  1.23052174 -0.89576072
  -0.31973742]
 [ 2.07428722 -2.05113012 -0.36039513 ...  1.23052174  0.73289513
  -0.31973742]
 [ 0.28675066 -0.27545206 -1.00611988 ...  1.23052174  0.73289513
  -0.31973742]
 ...
 [ 0.28675066 -0.27545206  0.93105437 ... -0.80592301  0.73289513
  -0.31973742]
 [-0.30909486  0.16846746 -0.03753275 ... -0.80592301  0.73289513
  -0.31973742]
 [ 3.86182378 -1.60721061 -0.6832575  ...  1.23052174 -0.89576072
  -0.31973742]]
Model 1 accuracy score: 0.9405


In [165]:
# Decision Tree Classification

clf = tree.DecisionTreeClassifier()
clf = clf.fit(ufs_train, y_train)
print(clf.score(ufs_test, y_test))

0.88475
